In [1]:
import os
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
findspark.init()

In [3]:
scala_version = "2.12"
spark_version = "3.5.0"

packages = f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}'
spark = SparkSession.builder. \
                    appName("KafkaSparkConnect"). \
                    config("spark.jars.packages", packages). \
                    getOrCreate()

In [4]:
BOOT_STRAP = "localhost:9092"
TOPIC = "kafkademo1"

kafka_df = spark.readStream. \
                format("kafka"). \
                option("kafka.bootstrap.servers", BOOT_STRAP). \
                option("subscribe", TOPIC). \
                load()\
                .withColumn("value", regexp_replace(col("value").cast("string"), "\\\\", "")) \
                .withColumn("value", regexp_replace(col("value"), "^\"|\"$", ""))

In [5]:
# base_df = kafka_df. \
#         selectExpr("CAST(value AS STRING)")

# base_df.printSchema()

In [6]:
sample_schema = (
        StructType()
        .add("id",StringType())
        .add("first_name", StringType())
        .add("last_name", StringType())
        .add("company", StringType())
        .add("role", StringType())
    )


fin_df = kafka_df.select(
        from_json(col("value"), sample_schema).alias("sample")
    ).select("sample.*")

In [7]:
fin_df.withColumn("id",col("id").cast("int"))



DataFrame[id: int, first_name: string, last_name: string, company: string, role: string]

In [8]:
fin_df.printSchema()


root
 |-- id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- company: string (nullable = true)
 |-- role: string (nullable = true)



In [9]:
url="jdbc:postgresql://localhost:5432/scd_2"
# source_name='source_scd'
# dest_name='dest_scd'
username='postgres'
password='Vish@08'

In [11]:
def foreach_batch_function(df,id):
    df.write.mode("append") \
      .format("jdbc") \
      .option("driver","org.postgresql.Driver") \
      .option("url", url) \
      .option("dbtable", "kafka_stream") \
      .option("user", username) \
      .option("password", password) \
      .save()
    pass
# 
fin_df \
    .writeStream \
    .outputMode("append") \
    .foreachBatch(foreach_batch_function)\
    .start() \
    .awaitTermination()


#ROUGH PRACTICE

In [ ]:
query = fin_df.writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", "C:/Users/SpringMl/kafka/my_output") \
    .option("checkpointLocation", "C:/Users/SpringMl/kafka/my_check") \
    .trigger(processingTime="1 minute") \
    .start()

query.awaitTermination()

In [ ]:
# from pyspark.sql.streaming import Trigger
# import org.apache.spark.sql.streaming.Trigger

In [ ]:
fin_df.writeStream.format("jdbc"). \
                    option("url", url). \
                    option("driver", "org.postgresql.Driver"). \
                    option("dbtable", "kafka_stream"). \
                    option("user", username). \
                    option("password", password). \
                    outputMode("append").trigger(Trigger.ProcessingTime("1 minute")).start()

In [ ]:
{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}
{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PASEO COSTA DEL SUR","State":"PR"}
{"Zipcode":709,"ZipCodeType":"STANDARD","City":"BDA SAN LUIS","State":"PR"}
{"Zipcode":76166,"ZipCodeType":"UNIQUE","City":"CINGULAR WIRELESS","State":"TX"}
{"Zipcode":76177,"ZipCodeType":"STANDARD","City":"FORT WORTH","State":"TX"}
{"Zipcode":76177,"ZipCodeType":"STANDARD","City":"FT WORTH","State":"TX"}
{"Zipcode":704,"ZipCodeType":"STANDARD","City":"URB EUGENE RICE","State":"PR"}
{"Zipcode":85209,"ZipCodeType":"STANDARD","City":"MESA","State":"AZ"}
{"Zipcode":85210,"ZipCodeType":"STANDARD","City":"MESA","State":"AZ"}
{"Zipcode":32046,"ZipCodeType":"STANDARD","City":"HILLIARD","State":"FL"}

In [ ]:
spark.stop()